# 라이브러리 임포트

In [1]:
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager

from selenium import webdriver

from tqdm import tqdm
from app_store_scraper import AppStore

import time
import pandas as pd
import numpy as np
import random
from datetime import datetime


# 수집기간 및 저장경로 지정

In [2]:
start_date = datetime(2024, 1, 1)

## PlayStore 리뷰 수집

In [3]:
from google_play_scraper import Sort, reviews_all

In [15]:
# result = reviews_all(
#     'com.kakaopay.app',
#     # 'com.naverfin.payapp',
#     # 'viva.republica.toss',
#     sleep_milliseconds=0, 
#     country='kr',
#     lang='ko',
#     sort=Sort.NEWEST, 
# )

# df = pd.DataFrame(result)

pay_google = ['com.kakaopay.app', 'com.naverfin.payapp', 'viva.republica.toss']
play_store_reviews = pd.DataFrame()
for i in tqdm(pay_google):
    result = reviews_all(
        i,
        sleep_milliseconds=0, 
        country='kr',
        lang='ko',
        sort=Sort.NEWEST,
        )
    temp = pd.DataFrame(result)
    play_store_reviews = pd.concat([play_store_reviews,temp], axis=0)

play_store_reviews


100%|██████████| 3/3 [01:26<00:00, 28.70s/it]


,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,476caf72-5e0a-4749-a01d-d874d324e61f,Google 사용자,https://play-lh.googleusercontent.com/EGemoI2N...,내 개인정보가 알리페이로 갔다 뿌슝삐숑?,1,0,None,2025-01-30 12:48:39,None,NaT,None
1,a4f2b74b-06b3-4f11-94de-3066e50376f0,Google 사용자,https://play-lh.googleusercontent.com/EGemoI2N...,Heaven is helping you🪙,5,0,3.1.6,2025-01-30 09:04:00,None,NaT,3.1.6
2,e7a8d4e4-e721-4ac4-a6a6-858c1d59f522,Google 사용자,https://play-lh.googleusercontent.com/EGemoI2N...,주식 모으기 매일 매수설정했는데 매수단가가 너무 높을때 구매돼요 수수료 무료라곤 하...,1,0,3.5.5,2025-01-30 00:15:37,None,NaT,3.5.5
3,c109bfde-36bc-441d-b565-4a29fbda77e4,Google 사용자,https://play-lh.googleusercontent.com/EGemoI2N...,받은거 취소도 어떻게 하는지 모르겠고 갑자기 받아지고 불편합니다. 광고하고 이상한거...,1,0,None,2025-01-29 14:14:39,None,NaT,None
4,597a0cde-f89a-4d40-b110-e598b23e21e6,Google 사용자,https://play-lh.googleusercontent.com/EGemoI2N...,폰과 테블릿 둘중 하나만 쓸수있는게 너무 불편해요. 카카오페이 증권 주거래 기기를 ...,3,0,3.5.5,2025-01-29 08:30:47,None,NaT,3.5.5
...,...,...,...,...,...,...,...,...,...,...,...
113106,71dc0d31-b5fd-405b-bac0-ef83feacc7a9,Google 사용자,https://play-lh.googleusercontent.com/EGemoI2N...,None,5,0,1.0.1,2014-03-12 21:06:32,None,NaT,1.0.1
113107,9e96512b-6edb-421e-b18d-a7fd8511898f,Google 사용자,https://play-lh.googleusercontent.com/EGemoI2N...,None,5,0,1.0.1,2014-03-12 19:36:04,None,NaT,1.0.1
113108,93db8a4f-6e21-4dde-98f1-47ad7604eea9,Google 사용자,https://play-lh.googleusercontent.com/EGemoI2N...,None,5,0,1.0.0,2014-03-12 16:58:07,None,NaT,1.0.0
113109,7b66f00c-b483-4647-9972-aad468d602bc,Google 사용자,https://play-lh.googleusercontent.com/EGemoI2N...,None,5,0,1.0.0,2014-03-12 16:50:46,None,NaT,1.0.0


## AppStore 리뷰 수집

In [21]:
def Extract_reviews(app_name:str, app_id:int): 
  pay = AppStore(country='kr', app_name=app_name, app_id=app_id)
  pay.review()
  pay_reviews = pd.DataFrame(pay.reviews)
  pay_reviews['pay'] = app_name
  return pay_reviews

In [22]:
pay_app = {'kakaopay': 1464496236, 'naverpay': 1554807824, 'toss': 839333328}

app_store_reviews = pd.DataFrame()
for i in tqdm(pay_app):
  pay_reviews = Extract_reviews(i, pay_app[i])
  app_store_reviews= pd.concat([app_store_reviews, pay_reviews], axis=0)

print('Done..')

  0%|          | 0/3 [00:00<?, ?it/s]2025-01-31 17:40:08,645 [INFO] Base - Initialised: AppStore('kr', 'kakaopay', 1464496236)
2025-01-31 17:40:08,646 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/kr/app/kakaopay/id1464496236
2025-01-31 17:40:13,740 [INFO] Base - [id:1464496236] Fetched 340 reviews (340 fetched in total)
2025-01-31 17:40:16,971 [INFO] Base - [id:1464496236] Fetched 1550 reviews (1550 fetched in total)
 33%|███▎      | 1/3 [00:08<00:16,  8.45s/it]2025-01-31 17:40:17,042 [INFO] Base - Initialised: AppStore('kr', 'naverpay', 1554807824)
2025-01-31 17:40:17,043 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/kr/app/naverpay/id1554807824
2025-01-31 17:40:22,891 [INFO] Base - [id:1554807824] Fetched 180 reviews (180 fetched in total)
2025-01-31 17:40:35,727 [INFO] Base - [id:1554807824] Fetched 480 reviews (480 fetched in total)
2025-01-31 17:40:41,211 [INFO] Base - [id:1554807824] Fetched 720 reviews (720 fetched in total)
2025-01-31 17

Done..


In [23]:
app_store_reviews

,date,developerResponse,review,rating,isEdited,title,userName,pay
0,2023-04-12 11:11:59,"{'id': 35986362, 'body': '안녕하세요, 카카오페이 앱을 이용해주...",주식때문에 하루에도 몇번? 몇십번을 열어보는 어플인데 사실 불편한 점이나 버퍼링이 ...,5,False,이번 업데이트 만족,yuni3yuni,kakaopay
1,2023-07-25 07:30:55,"{'id': 38034211, 'body': '안녕하세요, 카카오페이 서비스를 이용...",카카오페이는 결제나 송금만 이용했었는데 주식 되고 나서 접근성도 좋고 주식 알림 걸...,5,False,주식 관련 알림과 정보 주는거 너무 좋아요!,뉸뉴냔냐냐히,kakaopay
2,2023-05-17 00:17:46,"{'id': 36608464, 'body': '안녕하세요, 와플 왕팬님. 서비스 이...","[주식 선물하기] > [누구에게 주식을 선물할까요?]를 누르면 ""일시적인 오류가 발...",5,False,주식 선물하기 기능 오류,와플 왕팬,kakaopay
3,2021-09-27 16:00:06,"{'id': 25381057, 'body': '안녕하세요, j_jay_님. 카카오페...","이번 개편 버전 정말 대박입니다 😍 홈도 필요한 기능으로 설정할 수 있고 팁, 혜택...",5,False,금융 플랫폼 끝판왕이네요!,j_jay_,kakaopay
4,2021-06-22 20:10:42,"{'id': 23553595, 'body': '안녕하세요, bwidhvrc님. 카카...",카카오페이 결제화면에서 기본은 카카오페이머니로 돼있어서 결제수단을 카드로 변경해야할...,3,False,결제칸에서 카드 선택할때 위아래로 쓸어넘겨서 바꿀수있게 해주세요,bwidhvrc,kakaopay
...,...,...,...,...,...,...,...,...
7535,2024-12-15 04:01:27,"{'id': 49018059, 'body': '안녕하세요. BUzzcutseason...",위젯에 만보기가 자꾸 없어집니다!!! 업뎃 하면 다시 돌아오고... 이유가 뭔가요..😩😩,3,False,만보기가 자꾸 없어져요..💔💔,BUzzcutseason,toss
7536,2024-11-02 14:31:52,"{'id': 47946692, 'body': '안녕하세요. 문건우짱님, 토스팀입니다...",아니 뒷면에 카드대라고해서 몇번을 댔고\n전원 끄면서까지 했는데 안되는건 뭡니까? ...,1,False,뭐함?ㅋㅋ,문건우짱,toss
7537,2024-10-31 09:53:29,"{'id': 47905697, 'body': '안녕하세요. ㅊㄱㅂㅋ님, 토스팀입니다...",무슨 앱을 들어갔다가 나오기만 하면 초기화가 돼서 인증서 발급을 못하네 진짜 사람 ...,1,False,토스 진짜 미친 거 아닌가,ㅊㄱㅂㅋ,toss
7538,2024-09-26 05:48:09,"{'id': 47064724, 'body': '안녕하세요. 토스팀입니다. 친구와 함...",토스 닉네임으로 변경할 수 있게좀 해주세요 이름 다 뜨고..지나가는 모르는 사람이 ...,2,False,닉네임 변경,귀여운 축아,toss


# 스토어별 리뷰 구분 컬럼 추가

In [ ]:
play_store_reviews['store'] = 'google'
app_store_reviews['store'] = 'app'

In [32]:
play_store_reviews.head(2)

,reviewId,userName,userImage,review,score,thumbsUpCount,reviewCreatedVersion,date,replyContent,repliedAt,appVersion
0,476caf72-5e0a-4749-a01d-d874d324e61f,Google 사용자,https://play-lh.googleusercontent.com/EGemoI2N...,내 개인정보가 알리페이로 갔다 뿌슝삐숑?,1,0,None,2025-01-30 12:48:39,None,NaT,None
1,a4f2b74b-06b3-4f11-94de-3066e50376f0,Google 사용자,https://play-lh.googleusercontent.com/EGemoI2N...,Heaven is helping you🪙,5,0,3.1.6,2025-01-30 09:04:00,None,NaT,3.1.6


In [33]:
app_store_reviews.head(2)

,date,developerResponse,review,score,isEdited,title,userName,pay
0,2023-04-12 11:11:59,"{'id': 35986362, 'body': '안녕하세요, 카카오페이 앱을 이용해주...",주식때문에 하루에도 몇번? 몇십번을 열어보는 어플인데 사실 불편한 점이나 버퍼링이 ...,5,False,이번 업데이트 만족,yuni3yuni,kakaopay
1,2023-07-25 07:30:55,"{'id': 38034211, 'body': '안녕하세요, 카카오페이 서비스를 이용...",카카오페이는 결제나 송금만 이용했었는데 주식 되고 나서 접근성도 좋고 주식 알림 걸...,5,False,주식 관련 알림과 정보 주는거 너무 좋아요!,뉸뉴냔냐냐히,kakaopay


In [31]:
play_store_reviews = play_store_reviews.rename(columns= {'at':'date', 'content':'review'})
app_store_reviews = app_store_reviews.rename(columns= {'rating':'score'})